<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Codes/ask_pubmed_tweet_overlap_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

In [0]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Training%20set/AskAPatient.fold-0.train.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Validation%20set/AskAPatient.fold-0.validation.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Testing%20set/AskAPatient_overlap_20.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [15]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

   0                1
0  0             Fear
1  0             Fear
2  0             Fear
3  0  scared to death
4  1   Abdominal rash


In [17]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [19]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [20]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [22]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'diclofenac',
 'was',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'i',
 'be',
 'mg',
 'disease',
 'group',
 'study',
 'treatment',
 'are',
 'pain',
 'after',
 'not',
 'amp',
 'an',
 'it',
 'p',
 'have',
 'release',
 'gluten',
 'free',
 'we',
 'than',
 'all',
 'effect',
 'no',
 'compared',
 'new',
 'more',
 'using',
 'effects',
 'groups',
 'your',
 'our',
 'two',
 'between',
 'its',
 'both',
 'significantly',
 'my',
 'h',
 'about',
 'out',
 'these',
 'inflammatory',
 'time',
 'significant',
 'but',
 'drugs',
 'used',
 'use',
 'which',
 'day',
 'can',
 'voltaren',
 'anti',
 'dose',
 'results',
 'has',
 'potassium',
 's',
 'also',
 'placebo',
 'what',
 'glutenfree',
 'do',
 'may',
 'nt',
 'one',
 'efficacy',
 'there',
 'm',
 'celiac',
 'treated',
 'ds',
 'w',

In [23]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [24]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


In [0]:
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [0]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [0]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [0]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

In [0]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)